In [3]:
%pip uninstall sagemaker -y

Found existing installation: sagemaker 2.165.0
Uninstalling sagemaker-2.165.0:
  Successfully uninstalled sagemaker-2.165.0
Note: you may need to restart the kernel to use updated packages.


In [4]:
%pip install sagemaker==2.132.0

  Using cached sagemaker-2.132.0-py2.py3-none-any.whl
  Using cached attrs-22.2.0-py3-none-any.whl (60 kB)
  Using cached importlib_metadata-4.13.0-py3-none-any.whl (23 kB)
  Using cached urllib3-1.26.16-py2.py3-none-any.whl (143 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.0.3
    Uninstalling urllib3-2.0.3:
      Successfully uninstalled urllib3-2.0.3
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 6.6.0
    Uninstalling importlib-metadata-6.6.0:
      Successfully uninstalled importlib-metadata-6.6.0
  Attempting uninstall: attrs
    Found existing installation: attrs 23.1.0
    Uninstalling attrs-23.1.0:
      Successfully uninstalled attrs-23.1.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pytest-astropy 0.8.0 requires pytest-cov>=2.0, which is not installed.
pytest-astrop

In [5]:
import sagemaker
sagemaker.__version__

'2.132.0'

In [6]:
!pip install -q opencv-python torchvision facenet_pytorch torch gaze-tracking opencv-python ffmpeg-python

In [7]:
import torch
torch.__version__

'1.13.1+cu117'

In [8]:
import pandas as pd
import json
import boto3
import pathlib
import io
import sagemaker
from time import gmtime, strftime, sleep
from sagemaker.deserializers import CSVDeserializer
from sagemaker.serializers import CSVSerializer

from sagemaker.workflow.pipeline_context import PipelineSession
from sagemaker.xgboost.estimator import XGBoost
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.processing import (
    ProcessingInput, 
    ProcessingOutput, 
    ScriptProcessor
)
from sagemaker.inputs import TrainingInput

from sagemaker.workflow.pipeline import Pipeline
from sagemaker.workflow.steps import (
    ProcessingStep, 
    TrainingStep, 
    CreateModelStep
)
from sagemaker.workflow.check_job_config import CheckJobConfig
from sagemaker.workflow.parameters import (
    ParameterInteger, 
    ParameterFloat, 
    ParameterString, 
    ParameterBoolean
)
from sagemaker.workflow.clarify_check_step import (
    ModelBiasCheckConfig, 
    ClarifyCheckStep, 
    ModelExplainabilityCheckConfig
)
from sagemaker import Model
from sagemaker.inputs import CreateModelInput
from sagemaker.workflow.model_step import ModelStep
from sagemaker.workflow.fail_step import FailStep
from sagemaker.workflow.conditions import (
    ConditionGreaterThan,
    ConditionGreaterThanOrEqualTo
)
from sagemaker.workflow.pipeline_experiment_config import PipelineExperimentConfig
from sagemaker.workflow.properties import PropertyFile
from sagemaker.workflow.condition_step import ConditionStep
from sagemaker.workflow.functions import (
    Join,
    JsonGet
)
from sagemaker.workflow.lambda_step import (
    LambdaStep,
    LambdaOutput,
    LambdaOutputTypeEnum,
)
from sagemaker.lambda_helper import Lambda

from sagemaker.model_metrics import (
    MetricsSource, 
    ModelMetrics, 
    FileSource
)
from sagemaker.drift_check_baselines import DriftCheckBaselines

from sagemaker.image_uris import retrieve

In [9]:
boto_session = boto3.Session()
region = boto_session.region_name
bucket_name = sagemaker.Session().default_bucket()
bucket_prefix = "emo-video/resnet"  
sm_session = sagemaker.Session()
sm_client = boto_session.client("sagemaker")
sm_role = sagemaker.get_execution_role()

initialized = True

print(sm_role)

arn:aws:iam::567821811420:role/service-role/AmazonSageMaker-ExecutionRole-20230619T084765


In [10]:
%store bucket_name
%store bucket_prefix
%store sm_role
%store region
%store initialized

Stored 'bucket_name' (str)
Stored 'bucket_prefix' (str)
Stored 'sm_role' (str)
Stored 'region' (str)
Stored 'initialized' (bool)


In [11]:
import os

NOTEBOOK_METADATA_FILE = "/opt/ml/metadata/resource-metadata.json"
domain_id = None

if os.path.exists(NOTEBOOK_METADATA_FILE):
    with open(NOTEBOOK_METADATA_FILE, "rb") as f:
        domain_id = json.loads(f.read()).get('DomainId')
        print(f"SageMaker domain id: {domain_id}")

%store domain_id

SageMaker domain id: d-ivd5gnez0yil
Stored 'domain_id' (str)


In [12]:
experiment_name = f"emo-video-experiment-{strftime('%d-%H-%M-%S', gmtime())}"
%store experiment_name

Stored 'experiment_name' (str)


In [13]:
%store -r 

%store

try:
    initialized
except NameError:
    print("+++++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] YOU HAVE TO RUN 00-start-here notebook   ")
    print("+++++++++++++++++++++++++++++++++++++++++++++++++")

Stored variables and their in-db values:
baseline_s3_url                        -> 's3://sagemaker-eu-central-1-567821811420/emovideo
bucket_name                            -> 'sagemaker-eu-central-1-567821811420'
bucket_prefix                          -> 'emo-video/resnet'
domain_id                              -> 'd-ivd5gnez0yil'
evaluation_s3_url                      -> 's3://sagemaker-eu-central-1-567821811420/emovideo
experiment_name                        -> 'emo-video-experiment-29-23-19-59'
initialized                            -> True
input_s3_url                           -> 's3://sagemaker-eu-central-1-567821811420/from-ide
model_package_group_name               -> 'emo-video-model-group'
output_s3_url                          -> 's3://sagemaker-eu-central-1-567821811420/emovideo
prediction_baseline_s3_url             -> 's3://sagemaker-eu-central-1-567821811420/emovideo
region                                 -> 'eu-central-1'
sm_role                                -> 'arn:

In [14]:
import pandas as pd

df_data = pd.read_csv("./data/fer2013.csv", sep=";")

pd.set_option("display.max_columns", 500)  # View all of the columns
df_data  # show first 5 and last 5 rows of the dataframe

,"emotion,pixels,Usage"
0,"0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 1..."
1,"0,151 150 147 155 148 133 111 140 170 174 182 ..."
2,"2,231 212 156 164 174 138 161 173 182 200 106 ..."
3,"4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21..."
4,"6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 ..."
...,...
2118,"2,255 255 255 255 255 255 254 255 227 147 141 ..."
2119,"5,231 230 221 224 229 213 199 159 161 190 166 ..."
2120,"3,24 30 23 25 31 34 36 28 45 57 60 63 61 81 10..."
2121,"2,116 93 79 61 61 70 98 121 142 160 165 173 18..."


In [15]:
# Set names of pipeline objects
project = "emo-video"

pipeline_name = f"{project}-pipeline"
pipeline_model_name = f"{project}-model-resnet"
model_package_group_name = f"{project}-model-group"
endpoint_config_name = f"{project}-endpoint-config"
endpoint_name = f"{project}-endpoint"

In [16]:
# Set instance types and counts
process_instance_type = "ml.c5.xlarge"
train_instance_count = 1
train_instance_type = "ml.m5.xlarge"

In [17]:
# Init
boto_session = boto3.Session()
region = boto_session.region_name
bucket_name = sagemaker.Session().default_bucket()
bucket_prefix ="emovideo/resnet"  
sm_session = sagemaker.Session()
sm_client = boto_session.client("sagemaker")
sm_role = sagemaker.get_execution_role()

initialized =True
print(sm_role)

arn:aws:iam::567821811420:role/service-role/AmazonSageMaker-ExecutionRole-20230619T084765


In [18]:
# Set S3 urls for processed data
train_s3_url = f"s3://{bucket_name}/{bucket_prefix}/train"
validation_s3_url = f"s3://{bucket_name}/{bucket_prefix}/validation"
test_s3_url = f"s3://{bucket_name}/{bucket_prefix}/test"
baseline_s3_url = f"s3://{bucket_name}/{bucket_prefix}/baseline"

evaluation_s3_url = f"s3://{bucket_name}/{bucket_prefix}/evaluation"
prediction_baseline_s3_url = f"s3://{bucket_name}/{bucket_prefix}/prediction_baseline"

output_s3_url = f"s3://{bucket_name}/{bucket_prefix}/output"

In [19]:
%store train_s3_url
%store validation_s3_url
%store test_s3_url
%store baseline_s3_url
%store model_package_group_name
%store evaluation_s3_url
%store prediction_baseline_s3_url
%store output_s3_url

Stored 'train_s3_url' (str)
Stored 'validation_s3_url' (str)
Stored 'test_s3_url' (str)
Stored 'baseline_s3_url' (str)
Stored 'model_package_group_name' (str)
Stored 'evaluation_s3_url' (str)
Stored 'prediction_baseline_s3_url' (str)
Stored 'output_s3_url' (str)


In [20]:
try:
    input_s3_url
except NameError:      
    # If input_s3_url is not defined, upload the dataset to S3 and store the path
    input_s3_url = sagemaker.Session().upload_data(
        path="data/fer2013.csv",
        bucket=bucket_name,
        key_prefix=f"{bucket_prefix}/input"
    )
    print(f"Upload the dataset to {input_s3_url}")

    %store input_s3_url

In [21]:
# Set processing instance type
process_instance_type_param = ParameterString(
    name="ProcessingInstanceType",
    default_value=process_instance_type,
)

# Set training instance type
train_instance_type_param = ParameterString(
    name="TrainingInstanceType",
    default_value=train_instance_type,
)

# Set training instance count
train_instance_count_param = ParameterInteger(
    name="TrainingInstanceCount",
    default_value=train_instance_count
)

# Set model approval param
model_approval_status_param = ParameterString(
    name="ModelApprovalStatus",
    default_value="PendingManualApproval"
)

# Minimal threshold for model performance on the test dataset
test_score_threshold_param = ParameterFloat(
    name="TestScoreThreshold", 
    default_value=0.5
)

# Set S3 url for input dataset
input_s3_url_param = ParameterString(
    name="InputDataUrl",
    default_value=input_s3_url,
)

In [22]:
session = PipelineSession()

Preprocess step

In [23]:
%%writefile preprocessing.py

import pandas as pd
import numpy as np
import argparse
import os

def _parse_args():
    
    parser = argparse.ArgumentParser()
    # Data, model, and output directories
    # model_dir is always passed in from SageMaker. By default this is a S3 path under the default bucket.
    parser.add_argument('--filepath', type=str, default='/opt/ml/processing/input/')
    parser.add_argument('--filename', type=str, default='fer2013.csv')
    parser.add_argument('--outputpath', type=str, default='/opt/ml/processing/output/')
    
    return parser.parse_known_args()


if __name__=="__main__":
    # Process arguments
    args, _ = _parse_args()
    all_data= pd.read_csv(os.path.join(args.filepath, args.filename), sep=";")
    groups = [g for _, g in all_data.groupby('Usage')]
    training_data = groups[2]
    validation_data = groups[1]
    testing_data = groups[0]
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    label_names = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
    def make_dataloader(data, batch_size, shuffle):
        images, labels = data['pixels'], data['emotion']
        images = np.array([np.fromstring(image, np.uint8, sep=' ') for image in images]) / 255.0 # normalizing data to be between 0 and 1
        images = torch.FloatTensor(images.reshape(images.shape[0], 1, 48, 48)).to(device) # 1 color channel, 48x48 images
        dataset = torch.utils.data.TensorDataset(images, torch.LongTensor(np.array(labels)).to(device))
        return torch.utils.data.DataLoader(dataset=dataset, batch_size=batch_size, shuffle=shuffle)
    train_loader = make_dataloader(training_data, 100, True)
    valid_loader = make_dataloader(validation_data, 100, False)
    
    

Overwriting preprocessing.py


In [24]:
sklearn_processor = SKLearnProcessor(
        framework_version="0.23-1",
        role=sm_role,
        instance_type=process_instance_type_param,
        instance_count=1,
        base_job_name=f"{pipeline_name}/preprocess",
        sagemaker_session=session,
    )
    
processing_inputs=[
    ProcessingInput(source=input_s3_url_param, destination="/opt/ml/processing/input")
]

processing_outputs=[
    ProcessingOutput(output_name="train_data", source="/opt/ml/processing/output/train", 
                     destination=train_s3_url),
    ProcessingOutput(output_name="validation_data", source="/opt/ml/processing/output/validation",
                     destination=validation_s3_url),
    ProcessingOutput(output_name="test_data", source="/opt/ml/processing/output/test",
                     destination=test_s3_url),
    ProcessingOutput(output_name="baseline_data", source="/opt/ml/processing/output/baseline", 
                     destination=baseline_s3_url),
]

processor_args = sklearn_processor.run(
    inputs=processing_inputs,
    outputs=processing_outputs,
    code='preprocessing.py',
    # arguments = ['arg1', 'arg2'],
)
    
# Define processing step
step_process = ProcessingStep(
    name=f"{pipeline_name}-preprocess-data",
    step_args=processor_args,
)

The input argument instance_type of function (sagemaker.image_uris.retrieve) is a pipeline variable (<class 'sagemaker.workflow.parameters.ParameterString'>), which is not allowed. The default_value of this Parameter object will be used to override it. Please make sure the default_value is valid.
/opt/conda/lib/python3.7/site-packages/sagemaker/workflow/pipeline_context.py:261: UserWarning: Running within a PipelineSession, there will be No Wait, No Logs, and No Job being started.
  UserWarning,


Training step

In [33]:
%%writefile pyorchtrain.py
from torch import optim
from torchvision import models
import torch.nn as nn
import random
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
import torch.utils.data
import matplotlib.pyplot as plt

random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.backends.cudnn.deterministic = True

def adjust_model(model):
    model.conv1 = nn.Conv2d(1, 64, model.conv1.kernel_size, model.conv1.stride, model.conv1.padding, bias=False)
    model.fc = nn.Linear(model.fc.in_features, 7, bias=False)
    return model

def train_model(model, criterion, optimizer, data_loader, eval_loader):
    model = model.to(device)
    test_accuracy_history = []
    test_loss_history = []
    for epoch in range(epochs):
        print(f"Training epoch {epoch}")
        
        model.train()
        for data, labels in data_loader:
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, labels)
            loss.backward()
            optimizer.step()
            
        accuracy, loss = eval_model(model, eval_loader, criterion)
        print(f"Acc: {accuracy}, loss: {loss}")
        test_accuracy_history.append(accuracy)
        test_loss_history.append(loss)
    return test_accuracy_history, test_loss_history

def eval_model(model, data_loader, criterion):
    model.eval()
    with torch.no_grad():
        accuracy = 0
        loss = 0
        for data, labels in data_loader:
            output = model(data)
            _, preds = torch.max(output.data, 1)
            equals = (preds == labels).cpu()
            accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
            loss += criterion(output, labels).data.cpu()
        return accuracy/len(data_loader), loss/len(data_loader)


Writing pyorchtrain.py


In [29]:
epochs = 10
learning_rate = 0.1

In [30]:
from sagemaker.pytorch.estimator import PyTorch
import torch

In [38]:
pytorch_estimator = PyTorch('train.py',
                            instance_type='ml.p3.2xlarge',
                            instance_count=1,
                            framework_version='1.8.0',
                            py_version='py3',
                            role = sm_role,
                            hyperparameters = {'epochs': 20, 'batch-size': 64, 'learning-rate': 0.1})
## need to edit

# Extract the S3 URIs from the Properties object
train_data_s3_uri = step_process.properties.ProcessingOutputConfig.Outputs["train_data"].S3Output.S3Uri
validation_data_s3_uri = step_process.properties.ProcessingOutputConfig.Outputs["validation_data"].S3Output.S3Uri


training_args = pytorch_estimator.fit({'train':input_s3_url}, wait=True)

INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: pytorch-training-2023-06-29-23-53-10-845


2023-06-29 23:53:11 Starting - Starting the training job...
2023-06-29 23:53:36 Starting - Preparing the instances for training......
2023-06-29 23:54:34 Downloading - Downloading input data...
2023-06-29 23:54:54 Training - Downloading the training image...........................
2023-06-29 23:59:31 Training - Training image download completed. Training in progress...bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2023-06-29 23:59:52,054 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2023-06-29 23:59:52,088 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2023-06-29 23:59:52,091 sagemaker_pytorch_container.training INFO     Invoking user training script.
2023-06-29 23:59:52,378 sagemaker-training-toolkit INFO     Invoking user script
Training Env:
{
    "additional_framework_parameters": {},
    "channel_input_dirs": {
        "train": "/

In [ ]:
from sagemaker.processing import ProcessingOutput

# Define the processing output configuration
processing_output = ProcessingOutput(
    output_name="train_data",
    source="/path/to/output/directory",
    destination="{}/train".format(s3_bucket_uri),
)

In [57]:
step_train = TrainingStep(
    name=f"{pipeline_name}-train",
    estimator=pytorch_estimator,  # Pass the PyTorch estimator object
    inputs=input_s3_url,
)

Evaluation step

Register step

In [55]:
step_train

TrainingStep(name='emo-video-pipeline-train', display_name=None, description=None, step_type=<StepTypeEnum.TRAINING: 'Training'>, depends_on=None)

In [54]:
model = Model(
    image_uri='train.py',
    model_data=input_s3_url,
    name=f"emovideo",
    sagemaker_session=session,
    role=sm_role,
)

model_metrics = ModelMetrics(
    model_statistics=MetricsSource(
        s3_uri="{}/evaluation.json".format(
            step_train.arguments["ProcessingOutputConfig"]["Outputs"][0]["S3Output"]["S3Uri"]
        ),
        content_type="application/json",
    )
)

register_args = model.register(
    content_types=["text/csv"],
    response_types=["text/csv"],
    inference_instances=["ml.t2.medium", "ml.m5.xlarge", "ml.m5.large"],
    transform_instances=["ml.m5.xlarge", "ml.m5.large"],
    model_package_group_name=model_package_group_name,
    approval_status=model_approval_status_param,
    model_metrics=model_metrics,
)

step_register = ModelStep(
    name=f"{pipeline_name}-register",
    step_args=register_args
)

INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.


KeyError: 'ProcessingOutputConfig'

Pipeline constructer

In [ ]:
pipeline = Pipeline(
    name=pipeline_name,
    parameters=[
        process_instance_type_param,
        train_instance_type_param,
        train_instance_count_param,
        model_approval_status_param,
        test_score_threshold_param,
        input_s3_url_param,
    ],
    steps=[step_process, step_train, step_eval],
    sagemaker_session=session,
)

In [ ]:
# Create a new or update existing Pipeline
pipeline.upsert(role_arn=sm_role)

In [ ]:
pipeline_definition = json.loads(pipeline.describe()['PipelineDefinition'])
pipeline_definition

Execution pipeline

In [ ]:
execution = pipeline.start(
    parameters=dict(
        ProcessingInstanceType=process_instance_type,
        TrainingInstanceType=train_instance_type,
        TrainingInstanceCount=train_instance_count,
        ModelApprovalStatus="PendingManualApproval",
        TestScoreThreshold=0.75,
        InputDataUrl=input_s3_url
    )
)

In [ ]:
execution.list_steps()